In [6]:
import numpy as np
from Coursework.ANNModel import model as mod
from Coursework.ANNModel import data as d
from Coursework.PSO import pswarm as ps
from Coursework.PSO import psobehaviour as psh
from Coursework.PSO.history import PSOHistory
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
data = d.Data("./Data/1in_cubic.txt")
X = data.get_rows()
y = data.get_output()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=14)

In [8]:
# Define a model
model = mod.ANN()
model.set_training_input(X_train, y_train)
model.add(mod.Layer(4, activation="sigmoid"))
model.add(mod.Layer(1, activation="null"))
model.compile()


Model Compiled!


In [9]:
# create a pso object
inner_pso = ps.PSO(swarm_size=10, num_informants=6, boundary_policy=psh.BoundaryPolicy.RANDOMREINIT, termination_policy=[psh.TerminationPolicy.ITERATIONS], termination_args={'max_iter': 100}, alpha=0.61, beta=1.3, gamma=1.4, delta=1.3, epsilon=0.6, verbose=False)

In [10]:
# Define the search dimensions and fitness function of inner_pso
inner_pso.set_search_dimensions(model.dimension_vec())
inner_pso.set_fitness_fn(model.evaluate_fitness)

In [12]:
# Use PSO to optimise the hyperparameters of inner_pso
swarm_size = 10
iterations = 50

meta_pso = ps.PSO(swarm_size=swarm_size, num_informants=6, boundary_policy=psh.BoundaryPolicy.RANDOMREINIT, termination_policy=[psh.TerminationPolicy.ITERATIONS], termination_args={'max_iter': iterations}, alpha=0.61, beta=1.3, gamma=1.4, delta=1.3, epsilon=0.6, verbose=True)

In [13]:
model_record = PSOHistory(inner_pso, swarm_size, iterations)

meta_pso.set_search_dimensions(model_record.dimension_vec())
meta_pso.set_fitness_fn(model_record.evaluate_fitness)


In [ ]:
fitness = meta_pso.run()
print(fitness.location)
print(fitness.fitness)

D:\GitProjects\Bio-comp\Coursework\ANNModel\loss.py:5: RuntimeWarning: overflow encountered in square
  return np.mean((y - yhat)**2)
D:\GitProjects\Bio-comp\Coursework\ANNModel\activations.py:50: RuntimeWarning: overflow encountered in exp
  return np.exp((-z)**2)


In [ ]:
print(fitness.location)
print(fitness.fitness)

In [ ]:
Tanh_config = [95.63656981, 4.84026189,  1.49822568,  0.8405314,   0.78303066,  0.85137874,  0.85560595]